In [164]:
#imports
import math
import numpy as np
import random
import itertools
from random import choices
import cmath
BOLD = '\033[1m'
RESET = '\033[0m'


In [165]:
#new
powers               = [1,2,3]
num_antennas_BS      = 3
pop_size             = 10          #S
region_size          = 2           #R       

In [192]:
#Generating all scenarios in dbm
def generate_all_scenarios(powers, no_of_users):
    combinations = list(itertools.product(powers, repeat=no_of_users))
    return np.array(combinations)

In [193]:
#new
all_database = generate_all_scenarios(powers, num_antennas_BS)
for i, scenario in enumerate(all_database, start=1):
    print(f"{i}. {scenario}")

1. [1 1 1]
2. [1 1 2]
3. [1 1 3]
4. [1 2 1]
5. [1 2 2]
6. [1 2 3]
7. [1 3 1]
8. [1 3 2]
9. [1 3 3]
10. [2 1 1]
11. [2 1 2]
12. [2 1 3]
13. [2 2 1]
14. [2 2 2]
15. [2 2 3]
16. [2 3 1]
17. [2 3 2]
18. [2 3 3]
19. [3 1 1]
20. [3 1 2]
21. [3 1 3]
22. [3 2 1]
23. [3 2 2]
24. [3 2 3]
25. [3 3 1]
26. [3 3 2]
27. [3 3 3]


In [194]:
# # generate regions
# def get_random_scenarios(population, pop_size, region_size):
#     if region_size * pop_size > len(population):
#         raise ValueError("k * pop_size cannot be greater than the number of elements in the population.")
    
#     population_list = population.tolist()  #database
#     all_random_scenarios = []
    
#     for _ in range(pop_size):
#         random_scenarios = random.sample(population_list, region_size)
#         all_random_scenarios.append(random_scenarios)
#         for scenario in random_scenarios:
#             population_list.remove(scenario)
    
    
#     return all_random_scenarios

In [195]:
# #new
# REGIONS = np.array(get_random_scenarios(all_database, pop_size, region_size))
# print(*REGIONS, sep='\n')
# print(type(REGIONS))


[[2 3 1]
 [1 1 1]]
[[3 2 3]
 [3 1 3]]
[[1 3 3]
 [2 2 2]]
[[1 3 1]
 [3 1 2]]
[[1 2 2]
 [2 1 2]]
[[2 3 3]
 [3 3 3]]
[[2 1 1]
 [2 2 3]]
[[3 2 2]
 [1 2 3]]
[[1 2 1]
 [1 1 3]]
[[1 1 2]
 [1 3 2]]
<class 'numpy.ndarray'>


In [170]:
#@ generating population in dbm
def generate_random_solutions(powers, no_of_users, num_random_solutions):
    return np.random.choice(powers, size=(num_random_solutions, no_of_users))


In [171]:
#@ converting population to linear watt
def calculate_linear_trans_power(pop_size_dbm):
    result=(pow(10, -3) * np.power(10.0, np.array(pop_size_dbm, dtype="float") / 10))
    return result


In [172]:
# #new
# Lin_population = calculate_linear_trans_power(REGIONS)
# print(Lin_population)
# print(type(Lin_population))

In [173]:
def row_wise_sums(scenarios):
    return [[sum(row) for row in group] for group in scenarios]

In [174]:
# #new
# SUM_REGIONS_Lin = row_wise_sums(Lin_population)
# print(*SUM_REGIONS_Lin, sep='\n')

In [175]:
def min_values_in_groups(scenarios, sums):
    min_values_and_scenarios = []
    for group, group_sum in zip(scenarios, sums):
        min_value = min(group_sum)
        min_index = group_sum.index(min_value)
        min_scenario = group[min_index]
        min_values_and_scenarios.append((min_scenario, min_value))
    return min_values_and_scenarios

In [176]:
# #new
# pop = min_values_in_groups(REGIONS, SUM_REGIONS_Lin)
# print(*pop,sep='\n')

In [177]:
#@ fitness function  ==== linear_power_vect
def sum_linear_powers(Lin_population):
    return np.sum(Lin_population, axis=2)

In [178]:
# #new
# Lin_vector= sum_linear_powers(Lin_population)
# print(*Lin_vector,sep='\n')
# print(type(Lin_vector))

In [179]:
# # $ calculate the bit rate matrix
# def calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise):
#     a = np.array(linear_pop)
#     b = np.array(SUM_GAIN)
#     bit_rate_matrix = (D * np.log2(1 + ((a * b) / linear_noise)))
#     return bit_rate_matrix

In [180]:
#@ Select first half of the population
def selection_half_population(size_pop, pop_size_dbm, row_sums):
    L = size_pop // 2
    
    # Combine the population and their corresponding row sums
    help_fun = np.column_stack((pop_size_dbm, row_sums))

    # Sort the combined array in ascending order of row sums
    sorted_help_fun = help_fun[np.argsort(help_fun[:, -1])]
    
    # Select the top half of the population
    parent = sorted_help_fun[:L, :-1]
    parent_EE = sorted_help_fun[:L, -1]
    
    # Combine the selected parents and their corresponding row sums
    selected_parents = [[list(parent_row), parent_EE_value] for parent_row, parent_EE_value in zip(parent, parent_EE)]
    
    return selected_parents

In [181]:
# #new
# population = [scenario for scenario, _ in pop]
# print(*population,sep='\n')
# # parents= selection_half_population(pop_size, population, Lin_vector)
# # print(*parents,sep='\n')


In [182]:
def extract_powers_in_dbm(selected_parents):
    powers = [list(parent[0]) for parent in selected_parents]
    return powers

In [212]:
#@
def one_point_crossover_first_second(parents):
    offspring = []
    
    # Perform crossover between the first two parents
    parent1 = parents[0][0]
    parent2 = parents[1][0]
    crossover_point = random.randint(1, len(parent1) - 1)
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
    offspring.append(offspring1)
    offspring.append(offspring2)

    # Perform crossover between the second two parents
    parent3 = parents[2][0]
    parent4 = parents[3][0]
    crossover_point = random.randint(1, len(parent3) - 1)
    offspring3 = parent3[:crossover_point] + parent4[crossover_point:]
    offspring4 = parent4[:crossover_point] + parent3[crossover_point:]
    offspring.append(offspring3)
    offspring.append(offspring4)
    
    
    parent5 = parents[4][0]
    parent6 = parents[5][0]
    crossover_point = random.randint(1, len(parent5) - 1)
    offspring5 = parent5[:crossover_point] + parent6[crossover_point:]
    offspring6 = parent6[:crossover_point] + parent5[crossover_point:]
    offspring.append(offspring5)
    offspring.append(offspring6)
    
    parent7 = parents[6][0]
    parent8 = parents[7][0]
    crossover_point = random.randint(1, len(parent7) - 1)
    offspring7 = parent7[:crossover_point] + parent8[crossover_point:]
    offspring8 = parent8[:crossover_point] + parent7[crossover_point:]
    offspring.append(offspring7)
    offspring.append(offspring8)
    
    parent9 = parents[8][0]
    parent10 = parents[9][0]
    crossover_point = random.randint(1, len(parent9) - 1)
    offspring9 = parent9[:crossover_point] + parent10[crossover_point:]
    offspring10 = parent10[:crossover_point] + parent9[crossover_point:]
    offspring.append(offspring9)
    offspring.append(offspring10)
    
    

    return offspring





In [213]:
random.randint(1, 3)

3

In [214]:
def mutation(offspring, powers):
    mutated_offspring = []
    for child in offspring:
        mutated_child = child.copy()
        mutation_position = random.randint(0, len(child) - 1)
        new_gene = random.choice(powers)
        
        while new_gene == mutated_child[mutation_position]:
            new_gene = random.choice(powers)
        
        mutated_child[mutation_position] = new_gene
        mutated_offspring.append(mutated_child)
    return mutated_offspring



In [215]:
def concatenate_parents_and_offspring(selected_parents, mutated_offspring):
    combined_population = []

    # Add selected parents to the combined population
    for parent, parent_EE_value in selected_parents:
        combined_population.append(parent)

    # Add mutated offspring to the combined population
    for offspring in mutated_offspring:
        combined_population.append(np.array(offspring))

    # Convert the combined population list to a NumPy array
    combined_population = np.array(combined_population)

    return combined_population

# # Usage
# combined_population = concatenate_parents_and_offspring(selected_parents, mutated_offspring)
# print("Combined Population:")
# print(combined_population)


In [199]:
import random

# generate regions
def get_random_scenarios(population, pop_size, region_size):
    if region_size * pop_size > len(population):
        raise ValueError("k * pop_size cannot be greater than the number of elements in the population.")
    
    population_list = population.tolist()  #database
    all_random_scenarios = []
    
    for _ in range(pop_size):
        random_scenarios = random.sample(population_list, region_size)
        all_random_scenarios.append(random_scenarios)
        for scenario in random_scenarios:
            population_list.remove(scenario)
    
    
    return all_random_scenarios

In [209]:
powers               = [1,2,3,4]
num_antennas_BS      = 3
pop_size             = 20                                            
region_size          = 3                                          

D=10**6                                                            #bandwidth
N=-174 + 10 * math.log10(D)                                        #noise in dbm
linear_noise = calculate_linear_trans_power([N])
# generate the whole database

all_database = generate_all_scenarios(powers, num_antennas_BS)
# print("all database")
# print(*all_database, sep='\n')

# generate the region
REGIONS = get_random_scenarios(all_database, pop_size, region_size)
print("Regions")
print(*REGIONS, sep='\n')

# Q.population
REGIONS_Lin = calculate_linear_trans_power(REGIONS)
print("Regions in linear")
print(*REGIONS_Lin, sep='\n')

SUM_REGIONS_Lin = row_wise_sums(REGIONS_Lin)
print("sum of the regions")
print(*SUM_REGIONS_Lin, sep='\n')

pop = min_values_in_groups(REGIONS, SUM_REGIONS_Lin)
print("minimum values in region")
print(*pop, sep='\n')

population = [scenario for scenario, _ in pop]
print("dbm population")
print(population)

Regions
[[2, 4, 2], [4, 4, 4], [3, 3, 2]]
[[3, 4, 4], [4, 3, 1], [1, 2, 2]]
[[3, 3, 1], [2, 2, 2], [1, 3, 2]]
[[2, 3, 4], [1, 4, 2], [1, 2, 4]]
[[4, 1, 4], [3, 4, 1], [3, 2, 1]]
[[2, 4, 4], [3, 3, 3], [4, 4, 1]]
[[2, 2, 4], [1, 1, 1], [1, 4, 1]]
[[4, 2, 3], [3, 2, 4], [4, 2, 4]]
[[4, 3, 2], [3, 4, 3], [4, 1, 1]]
[[4, 3, 4], [1, 1, 3], [2, 2, 3]]
[[2, 4, 3], [4, 3, 3], [1, 3, 1]]
[[2, 4, 1], [2, 1, 4], [2, 3, 3]]
[[1, 2, 3], [1, 2, 1], [4, 2, 1]]
[[3, 2, 2], [2, 3, 1], [2, 1, 1]]
[[3, 3, 4], [1, 4, 3], [2, 1, 2]]
[[3, 2, 3], [4, 4, 2], [3, 1, 1]]
[[3, 1, 2], [4, 1, 3], [4, 2, 2]]
[[2, 2, 1], [3, 1, 4], [1, 3, 3]]
[[3, 1, 3], [1, 3, 4], [1, 4, 4]]
[[1, 1, 2], [1, 1, 4], [4, 4, 3]]
Regions in linear
[[0.00158489 0.00251189 0.00158489]
 [0.00251189 0.00251189 0.00251189]
 [0.00199526 0.00199526 0.00158489]]
[[0.00199526 0.00251189 0.00251189]
 [0.00251189 0.00199526 0.00125893]
 [0.00125893 0.00158489 0.00158489]]
[[0.00199526 0.00199526 0.00125893]
 [0.00158489 0.00158489 0.00158489]
 [0.

In [216]:
def sum_rows(data):
    return [sum(row) for row in data]

In [217]:
Lin_population = calculate_linear_trans_power(population)
print("Linear population")
print(*Lin_population, sep='\n')


Lin_vector     = sum_rows(Lin_population)
print("linear vector")
print(*Lin_vector, sep='\n')

parents        = selection_half_population(pop_size, population, Lin_vector)
print("parents")
print(*parents,sep='\n' )
    
best_power     = parents[0][1] 
best_scenario  = np.array(parents[0][0]).astype(int)
  
next_generation= np.array(extract_powers_in_dbm(parents)).astype(int)
print("just the powers")
print(next_generation)

crossover       = np.array(one_point_crossover_first_second(parents)).astype(int)
mutation1       = np.array(mutation(crossover, powers)).astype(int)
print("crossover", *crossover, sep='\n')
print("mutation",*mutation1, sep='\n')

Linear population
[0.00199526 0.00199526 0.00158489]
[0.00125893 0.00158489 0.00158489]
[0.00158489 0.00158489 0.00158489]
[0.00125893 0.00251189 0.00158489]
[0.00199526 0.00158489 0.00125893]
[0.00199526 0.00199526 0.00199526]
[0.00125893 0.00125893 0.00125893]
[0.00251189 0.00158489 0.00199526]
[0.00251189 0.00125893 0.00125893]
[0.00125893 0.00125893 0.00199526]
[0.00125893 0.00199526 0.00125893]
[0.00158489 0.00251189 0.00125893]
[0.00125893 0.00158489 0.00125893]
[0.00158489 0.00125893 0.00125893]
[0.00158489 0.00125893 0.00158489]
[0.00199526 0.00125893 0.00125893]
[0.00199526 0.00125893 0.00158489]
[0.00158489 0.00158489 0.00125893]
[0.00199526 0.00125893 0.00199526]
[0.00125893 0.00125893 0.00158489]
linear vector
0.005575417822398872
0.004428711796716394
0.004754679577383341
0.005355705035764861
0.00483908091922416
0.005985786944906638
0.0037767762353825016
0.006092041938939573
0.005029737255097915
0.0045131131385572135
0.004513113138557214
0.005355705035764861
0.0041027440160

In [221]:
def GA(population):
    Lin_population = calculate_linear_trans_power(population)
    Lin_vector     = sum_rows(Lin_population)
    parents        = selection_half_population(pop_size, population, Lin_vector)
    
    best_power     = parents[0][1] 
    best_scenario  = np.array(parents[0][0]).astype(int)
    
    next_generation= np.array(extract_powers_in_dbm(parents)).astype(int)
       
    crossover       = np.array(one_point_crossover_first_second(parents)).astype(int)
    mutation1       = np.array(mutation(crossover, powers)).astype(int)
#     print("crossover",crossover)
#     print("mutation1",mutation1)
    
    next_generation=  concatenate_parents_and_offspring(parents, mutation1)
    population     = next_generation

    return population,best_power,best_scenario

In [222]:
max_G = 100

for i in range(max_G):
    
    population,best_powerr,best_scenarioo = GA(population)
    print("generation:        ",i+1)
    #print("population",population)
    print("Best_scenario      ",best_scenarioo)
    print("Best_power         ",best_powerr)
    print("************************************************")
   

generation:         1
Best_scenario       [1 1 1]
Best_power          0.0037767762353825016
************************************************
generation:         2
Best_scenario       [1 1 1]
Best_power          0.0037767762353825016
************************************************
generation:         3
Best_scenario       [1 1 1]
Best_power          0.0037767762353825016
************************************************
generation:         4
Best_scenario       [1 1 1]
Best_power          0.0037767762353825016
************************************************
generation:         5
Best_scenario       [1 1 1]
Best_power          0.0037767762353825016
************************************************
generation:         6
Best_scenario       [1 1 1]
Best_power          0.0037767762353825016
************************************************
generation:         7
Best_scenario       [1 1 1]
Best_power          0.0037767762353825016
************************************************
generation:  